### Downloading all the dependencies that we need

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from google.colab import files

import io

print('Libraries imported.')

/bin/bash: conda: command not found
/bin/bash: conda: command not found
Libraries imported.


In [2]:
display(np)
display(pd)
display(json)
display(Nominatim)
display(json_normalize)
display(cm)
display(colors)
display(KMeans)
display(folium)
display(files)
display(io)

<module 'numpy' from '/usr/local/lib/python3.6/dist-packages/numpy/__init__.py'>

<module 'pandas' from '/usr/local/lib/python3.6/dist-packages/pandas/__init__.py'>

<module 'json' from '/usr/lib/python3.6/json/__init__.py'>

geopy.geocoders.osm.Nominatim

<function pandas.io.json._normalize._json_normalize>

...

<module 'matplotlib.colors' from '/usr/local/lib/python3.6/dist-packages/matplotlib/colors.py'>

sklearn.cluster._kmeans.KMeans

<module 'folium' from '/usr/local/lib/python3.6/dist-packages/folium/__init__.py'>

<module 'google.colab.files' from '/usr/local/lib/python3.6/dist-packages/google/colab/files.py'>

<module 'io' from '/usr/lib/python3.6/io.py'>

### Login to the FourSquare API

In [3]:
CLIENT_ID = 'TN2LARBAREO35LUKJP4IO2VBZ255I3SDPU5OQH5NCNHDWV2D' # your Foursquare ID
CLIENT_SECRET = 'AJK3AZPSCPUELGPMTRPVO2L3FHH3MCGN5AGOOQBFGUYQ30CO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TN2LARBAREO35LUKJP4IO2VBZ255I3SDPU5OQH5NCNHDWV2D
CLIENT_SECRET:AJK3AZPSCPUELGPMTRPVO2L3FHH3MCGN5AGOOQBFGUYQ30CO


### Use geopy library to get the latitude and longitude values of Singapore.

In [4]:
address = 'Singapore, SG'

geolocator = Nominatim(user_agent="sg_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singapore City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Singapore City are 1.3408630000000001, 103.83039182212079.


### Obtaining a DataFrame of the Neighbourhoods/Buildings of Singapore

In [5]:
#Here, we upload a CSV file that contains the list of Buildings and their co-ordinates.
#We will use these co-ordinates to call the FourSquare API to get the list of Venues and their Categoris for each building.
uploaded = files.upload()
display("Uploaded files are")
display(uploaded)

Saving Singapore Neighbourhoods.csv to Singapore Neighbourhoods.csv


'Uploaded files are'

{'Singapore Neighbourhoods.csv': b"Neighbourhood,Latitude,Longitude\r\nAVA MARINE AQUACULTURE CENTRE,1.216165365,103.8492479\r\nMARINE AQUACULTURE CENTRE,1.216347843,103.8497779\r\nTROPICAL MARINE SCIENCE INSTITUTE,1.217175596,103.8502961\r\nMOSQUE,1.219284623,103.849158\r\nKUSU ISLAND KERAMAT SHRINE,1.222907608,103.8615824\r\nKUSU ISLAND ENV CONTROL UNIT,1.222964185,103.8610424\r\nKUSU ISLAND KIOSK 21,1.223212754,103.8605995\r\nKUSU ISLAND HAWKER CENTRE,1.223218083,103.8609168\r\nKUSU ISLAND SINGAPORE,1.223219337,103.8599069\r\nKUSU ISLAND CHINESE TEMPLE,1.223313279,103.8599281\r\nMASJID PULAU BUKOM,1.228827828,103.7675714\r\nLNG TERMINAL,1.236338449,103.670849\r\nSEASCAPE,1.239784946,103.8372771\r\nSEVEN PALMS SENTOSA COVE,1.240058485,103.8309363\r\nCAPE ROYALE,1.241285286,103.8401003\r\nSANDY ISLAND,1.242009597,103.8380776\r\nTURQUOISE,1.242963973,103.8371062\r\nMARINA COLLECTION,1.242974246,103.8391646\r\nTHE GREEN COLLECTION,1.243305681,103.8366556\r\nDBS SembCorp Marine,1.2434701

In [6]:
df_singapore_raw = pd.read_csv(io.BytesIO(uploaded["Singapore Neighbourhoods.csv"]))

In [7]:
display(df_singapore_raw.head())
display(df_singapore_raw.shape)

,Neighbourhood,Latitude,Longitude
0,AVA MARINE AQUACULTURE CENTRE,1.216165,103.849248
1,MARINE AQUACULTURE CENTRE,1.216348,103.849778
2,TROPICAL MARINE SCIENCE INSTITUTE,1.217176,103.850296
3,MOSQUE,1.219285,103.849158
4,KUSU ISLAND KERAMAT SHRINE,1.222908,103.861582


(16028, 3)

### Obtain a smaller sample of the dataset

In [8]:
#Here, we are selecting a smaller sample of the data for the sake of simplicity.
df_singapore = df_singapore_raw.sample(
n = 5000,
replace = False,
axis = 0)

In [9]:
df_singapore.shape

(5000, 3)

### Define a function to obtain the Nearby venues for all neighbourhoods of Singapore

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    id = 0
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
        
        try:
            # make the GET request
            #print("Getting the items for: " + name)
            get_result = requests.get(url).json()
            
            #"get_result" has a nested dictionary.
            #The dictionary keys are {"meta", "response": {"groups[{"type", "name", "items"}]", ...}}
            #We are interested in the "items" which is inside "groups"(a list) which is inside "response"(a dictionary)
            results = get_result["response"]["groups"][0]["items"]
            #print(" Successfully obtained the items for: " + name)
            print("Obtained result for " + str(id) + " item")
            id = id + 1
            #display(results)
        except KeyError:
            print("Error extracting items for " + str(id) + " item")
            id = id + 1
            #print("Skipping this name")
            continue
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    if(len(venues_list) == 0):
      print("Error: Could not get info for any venue")
      return None
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
singapore_venues = getNearbyVenues(names=df_singapore['Neighbourhood'],
                                   latitudes=df_singapore['Latitude'],
                                   longitudes=df_singapore['Longitude']
                                  )

In [15]:
#Checking the Shape of "singapore_venues"
display(singapore_venues.head())
display(singapore_venues.shape)

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,LITTLE GREENHOUSE @ S540 PTE LTD,1.374619,103.872573,Food More by Sheng Siong,1.375394,103.872028,Food Court
1,LITTLE GREENHOUSE @ S540 PTE LTD,1.374619,103.872573,Sheng Siong Supermarket,1.375399,103.871979,Supermarket
2,LITTLE GREENHOUSE @ S540 PTE LTD,1.374619,103.872573,Xiao Di Fried Prawn Noodle,1.370650,103.872142,Noodle House
3,LITTLE GREENHOUSE @ S540 PTE LTD,1.374619,103.872573,Guan Hock Tiong Eating House 源福中餐室 (Guan Hock ...,1.370519,103.872231,Asian Restaurant
4,LITTLE GREENHOUSE @ S540 PTE LTD,1.374619,103.872573,KPT Coffee Shop,1.373814,103.875381,Coffee Shop


(26839, 7)

###Download this CSV into the local machine.
####This ensures we can avoid the time-consuming API calls executed above the next time. We can simply upload this CSV again.

In [19]:
#Converting the "singapore_venues" into a CSV file and downloading it.
#This enables us to avoid having to call the Foursquare API time and again
csv_file = singapore_venues.to_csv("singapore_venues.csv")
files.download("singapore_venues.csv")

###Upload the CSV having all the Singapore Venues

In [ ]:
#Here we simply upload the "singapore_venues.csv" from our local machine.
#This ensures we can avoid running the above calls to the FourSquare API
uploaded = files.upload()
display("Uploaded files are")
display(uploaded)

###Explore the singapore_venues DataFrame

In [ ]:
display(singapore_venues.groupby('Neighbourhood').nunique())

In [ ]:
display(singapore_venues.groupby('Neighbourhood').count())

In [ ]:
display(singapore_venues['Venue Category'].unique())

In [ ]:
display('There are {} uniques categories.'.format(len(singapore_venues['Venue Category'].unique())))
display('There are {} uniques neighbourhoods.'.format(len(singapore_venues['Neighbourhood'].unique())))

'There are 406 uniques categories.'

'There are 946 uniques neighbourhoods.'

### Execute OneHot encoding on the data
OneHot encoding converts the data to a new format.

The new format will have the Venue Categories across the columns and the Neighbourhoods across the Rows.

A Boolean value indicates if that Neighbourhood has at least one Venue of that Category.


In [21]:
# one hot encoding
singapore_onehot = pd.get_dummies(singapore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
singapore_onehot['Neighbourhood'] = singapore_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [singapore_onehot.columns[-1]] + list(singapore_onehot.columns[:-1])
singapore_onehot = singapore_onehot[fixed_columns]

singapore_onehot.head()

,Neighbourhood,ATM,Accessories Store,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cable Car,Cafeteria,Café,Campground,Canal,Candy Store,Cantonese Restaurant,Casino,Cemetery,Cha Chaan Teng,Cheese Shop,Child Care Service,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Lab,College Residence Hall,College Stadium,College Theater,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cruise Ship,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lighthouse,Lighting Store,Lottery Retailer,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Event Space,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Peking Duck Restaurant,Pe

In [22]:
#Let's verify the shape of the One-hot encoded data
display(singapore_onehot.head())
display(singapore_onehot.shape)

,Neighbourhood,ATM,Accessories Store,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cable Car,Cafeteria,Café,Campground,Canal,Candy Store,Cantonese Restaurant,Casino,Cemetery,Cha Chaan Teng,Cheese Shop,Child Care Service,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Lab,College Residence Hall,College Stadium,College Theater,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cruise Ship,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lighthouse,Lighting Store,Lottery Retailer,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Event Space,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Peking Duck Restaurant,Pe

(26839, 402)

In [23]:
#Let's group the rows by Neighbourhood and calculate the mean frequency of occurrance of each venue category
singapore_grouped = singapore_onehot.groupby('Neighbourhood').mean().reset_index()
display(singapore_grouped.head())
display(singapore_grouped.shape)

,Neighbourhood,ATM,Accessories Store,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cable Car,Cafeteria,Café,Campground,Canal,Candy Store,Cantonese Restaurant,Casino,Cemetery,Cha Chaan Teng,Cheese Shop,Child Care Service,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Lab,College Residence Hall,College Stadium,College Theater,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cruise Ship,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Leather Goods Store,Light Rail Station,Lighthouse,Lighting Store,Lottery Retailer,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Event Space,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Peking Duck Restaurant,Pe

(949, 402)

###Playing around with the data
Feel free to skip this part

In [ ]:
#Let's get the top 5 venues per neighbourhood
num_top_venues = 5

for hood in singapore_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = singapore_grouped[singapore_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
#Define a function that receives a neighbourhood and a number 'n'. It shall return the top n venues per neighbourhood.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
#Let's call the return_most_common_venues() function defined above and return the 10 most popular venues per neighbourhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = singapore_grouped['Neighborhood']

for ind in np.arange(singapore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(singapore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0.0,Chinese Restaurant,Coffee Shop,Indian Restaurant,Asian Restaurant,American Restaurant,Dessert Shop,Massage Studio,Gastropub,Supermarket,Park
1,0.0,Coffee Shop,Café,Chinese Restaurant,Clothing Store,Noodle House,Asian Restaurant,Ice Cream Shop,Shopping Mall,Seafood Restaurant,Indian Restaurant
2,0.0,Farm,Pet Store,Café,Seafood Restaurant,Flower Shop,Garden Center,Coffee Shop,Spa,Farmers Market,Lake
3,0.0,Coffee Shop,Shopping Mall,Supermarket,Food Court,Smoke Shop,Bus Stop,Fast Food Restaurant,Field,English Restaurant,Escape Room
4,0.0,Park,Supermarket,Coffee Shop,Food Court,Fast Food Restaurant,Basketball Court,Seafood Restaurant,Convenience Store,Dance Studio,Deli / Bodega


### Let's start querying specific data points!

####Indian Restaurants

In [ ]:
#display(singapore_onehot.head())
#display(singapore_venues[singapore_venues["Neighbourhood"] == "YUGI GARDEN"].groupby(["Venue Category"]).nunique())
#display(singapore_grouped[singapore_grouped["Neighbourhood"] == "YUGI GARDEN"].head())
display(singapore_onehot[singapore_onehot["Indian Restaurant"] == 1].head())

,Neighbourhood,ATM,Accessories Store,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cable Car,Cafeteria,Café,Campground,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cemetery,Cha Chaan Teng,Cheese Shop,Child Care Service,Chinese Aristocrat Restaurant,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Auditorium,College Bookstore,College Cafeteria,College Gym,College Stadium,College Theater,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Cruise Ship,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Dongbei Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,English Restaurant,Escape Room,Event Space,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Fujian Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lighthouse,Lighting Store,Lingerie Store,Lottery Retailer,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Outlet Store,Paella Restaurant,Paintball Field,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,P

In [ ]:
#Let's find neighbourhoods with sizeable presence of Indian Restaurants!
#There are three venue categories for Indian Restaurants - "Indian Restaurant", "North Indian Restaurant" and "South Indian Restaurant".

#First, lets get a list of Neighbourhoods that have at least one Indian restaurant, along with the frequency for each category.
indian_restaurant_frequency = singapore_grouped[singapore_grouped["Indian Restaurant"] + singapore_grouped["North Indian Restaurant"] + singapore_grouped["South Indian Restaurant"] > 0]
indian_restaurant_frequency = indian_restaurant_frequency[["Neighbourhood", "Indian Restaurant", "North Indian Restaurant", "South Indian Restaurant"]]
indian_restaurant_frequency = indian_restaurant_frequency.rename(columns = {"Indian Restaurant": "Indian Restaurant Frequency",
                                                  "North Indian Restaurant": "North Indian Restaurant Frequency",
                                                  "South Indian Restaurant": "South Indian Restaurant Frequency"})
display("indian_restaurant_frequency is")
display(indian_restaurant_frequency.head())
display(indian_restaurant_frequency.shape)
print("\n")

#Then, we need to get the actual list of these Indian Restaurants for each of these neighbourhoods.
indian_restaurant = singapore_venues[singapore_venues["Neighbourhood"].isin(indian_restaurant_frequency["Neighbourhood"].tolist())]
indian_restaurant = indian_restaurant[indian_restaurant["Venue Category"].isin(["Indian Restaurant", "North Indian Restaurant", "South Indian Restaurant"])]
indian_restaurant = indian_restaurant[["Neighbourhood", "Venue", "Venue Category"]]

display("indian_restaurant is")
display(indian_restaurant.head())
display(indian_restaurant.shape)
print("\n")

#We now have two tables:-
#"indian_restaurant_frequency" is a list of Neighbourhoods that have at least one Venue that is Indian Restaurant. It also records the frequency of the Indian Restaurants.
#"indian_restaurant" is a list of the actual Venues for each Neighbourhood that belong to the category of Indian Restaurant.

#Now let's do a GROUP BY to get the unique count of Venues per Neighbourhood and Venue Category.
#This should tell us how many Indian restaurants exist per neighbourhood.
indian_restaurant_grouped = indian_restaurant.groupby(["Neighbourhood", "Venue Category"])
indian_restaurant_grouped = indian_restaurant_grouped.agg({"Venue": "nunique"})
indian_restaurant_grouped = indian_restaurant_grouped.reset_index()
indian_restaurant_grouped = indian_restaurant_grouped.rename(columns = {"Venue": "Total Venues"})

display("indian_restaurant_grouped is")
display(indian_restaurant_grouped.head())
display(indian_restaurant_grouped.shape)
print("\n")

#Now, "indian_restaunrant_frequency" will tell us the frequency of Indian Restaurants in a given Neighbourhood.
#"indian_restaurant_grouped" will tell us the actual count of Indian Restaurants in a given Neighbourhood.
#We can join them to get the entire information in a single table

indian_restaurant_final = pd.merge(
    left = indian_restaurant_grouped,
    right = indian_restaurant_frequency,
    how = "inner",
    on = "Neighbourhood"
)
display("indian_restaurant_final is")
display(indian_restaurant_final.head())
display(indian_restaurant_final.shape)
print("\n")

In [ ]:
#AWESOME!
#We now have a single table "indian_restaurant_final" that has all the data(well, most) we need.
#In order to show the actual Venue names, we can always go back to "indian_restaurant" table.

#Let's see see some Neighbourhoods with the most number of Indian Restaurants in terms of Count and Frequency.
display(indian_restaurant_final[indian_restaurant_final["Venue Category"] == "South Indian Restaurant"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display(indian_restaurant_final[indian_restaurant_final["Venue Category"] == "Indian Restaurant"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display(indian_restaurant_final[indian_restaurant_final["Venue Category"] == "North Indian Restaurant"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display(indian_restaurant_final.sort_values(
    by = ["Indian Restaurant Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display(indian_restaurant_final.sort_values(
    by = ["North Indian Restaurant Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display(indian_restaurant_final.sort_values(
    by = ["South Indian Restaurant Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

####Museums and Art Galleries

In [ ]:
#Let's find neighbourhoods with sizeable presence of Museums and Art Galleries!
#There are six venue categories for Indian Restaurants - “Art Museum”, “History Museum”, “Museum”, “Science Museum”, “Art Gallery” and “Performing Arts Venue”.

#First, lets get a list of Neighbourhoods that have at least one Museum, along with the frequency for each category.
museum_art_gallery_frequency = singapore_grouped[
singapore_grouped["Art Museum"] + singapore_grouped["History Museum"] + singapore_grouped["Museum"]
+ singapore_grouped["Science Museum"] + singapore_grouped["Art Gallery"] + singapore_grouped["Performing Arts Venue"] > 0]


museum_art_gallery_frequency = museum_art_gallery_frequency[["Neighbourhood", "Art Museum", "History Museum", "Museum", "Science Museum", "Art Gallery", "Performing Arts Venue"]]
museum_art_gallery_frequency = museum_art_gallery_frequency.rename(columns = {"Art Museum": "Art Museum Frequency",
                                                      "History Museum": "History Museum Frequency",
                                                      "Museum": "Museum Frequency",
                                                      "Science Museum": "Science Museum Frequency",
                                                      "Art Gallery": "Art Gallery Frequency",
                                                      "Performing Arts Venue": "Performing Arts Venue Frequency"})
display("museum_art_gallery_frequency is")
display(museum_art_gallery_frequency.head())
display(museum_art_gallery_frequency.shape)
print("\n")

#Then, we need to get the actual list of these Museums for each of these neighbourhoods.
museum_art_gallery = singapore_venues[singapore_venues["Neighbourhood"].isin(museum_art_gallery_frequency["Neighbourhood"].tolist())]
museum_art_gallery = museum_art_gallery[museum_art_gallery["Venue Category"].isin(["Art Museum", "History Museum", "Museum", "Science Museum", "Art Gallery", "Performing Arts Venue"])]
museum_art_gallery = museum_art_gallery[["Neighbourhood", "Venue", "Venue Category"]]

display("museum_art_gallery is")
display(museum_art_gallery.head())
display(museum_art_gallery.shape)
print("\n")

#We now have two tables:-
#"museum_art_gallery_frequency" is a list of Neighbourhoods that have at least one Venue that is a Museum/Art Gallery. It also records the frequency of the Museums/Galleries.
#"museum_art_gallery" is a list of the actual Venues for each Neighbourhood that belong to the category of Museum/Art Gallery.

#Now let's do a GROUP BY to get the unique count of Venues per Neighbourhood and Venue Category.
#This should tell us how many Museums/Galleires exist per neighbourhood.
museum_art_gallery_grouped = museum_art_gallery.groupby(["Neighbourhood", "Venue Category"])
museum_art_gallery_grouped = museum_art_gallery_grouped.agg({"Venue": "nunique"})
museum_art_gallery_grouped = museum_art_gallery_grouped.reset_index()
museum_art_gallery_grouped = museum_art_gallery_grouped.rename(columns = {"Venue": "Total Venues"})

display("museum_art_gallery_grouped is")
display(museum_art_gallery_grouped.head())
display(museum_art_gallery_grouped.shape)
print("\n")

#Now, "museum_art_gallery_frequency" will tell us the frequency of Museums/Galleires in a given Neighbourhood.
#"museum_art_gallery_grouped" will tell us the actual count of Museums/Galleires in a given Neighbourhood.
#We can join them to get the entire information in a single table

museum_art_gallery_final = pd.merge(
    left = museum_art_gallery_grouped,
    right = museum_art_gallery_frequency,
    how = "inner",
    on = "Neighbourhood"
)
display("museum_art_gallery_final is")
display(museum_art_gallery_final.head())
display(museum_art_gallery_final.shape)
print("\n")

In [ ]:
#AWESOME!
#"museum_art_gallery_frequency" is a list of Neighbourhoods that have at least one Venue that is a Museum/Art Gallery. It also records the frequency of the Museums/Galleries.
#"museum_art_gallery" is a list of the actual Venues for each Neighbourhood that belong to the category of Museum/Art Gallery.
#"museum_art_gallery_final" has all the data(well, most) we need.
#In order to show the actual Venue names, we can always go back to "museum_art_gallery" table.

#Let's see see some Neighbourhoods with the most number of Museum/Art Galleries in terms of Count.
display("Highest number of Art Museums are")
display(museum_art_gallery_final[museum_art_gallery_final["Venue Category"] == "Art Museum"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of History Museums are")
display(museum_art_gallery_final[museum_art_gallery_final["Venue Category"] == "History Museum"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of General Museums are")
display(museum_art_gallery_final[museum_art_gallery_final["Venue Category"] == "Museum"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Science Museums are")
display(museum_art_gallery_final[museum_art_gallery_final["Venue Category"] == "Science Museum"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Art Galleries are")
display(museum_art_gallery_final[museum_art_gallery_final["Venue Category"] == "Art Gallery"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Peroforming Arts Venues are")
display(museum_art_gallery_final[museum_art_gallery_final["Venue Category"] == "Performing Arts Venue"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")


#Let's see see some Neighbourhoods with the most number of Museum/Art Galleries in terms of Frequency.
display("Highest frequency of Art Museums are")
display(museum_art_gallery_final.sort_values(
    by = ["Art Museum Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of History Museums are")
display(museum_art_gallery_final.sort_values(
    by = ["History Museum Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of General Museums are")
display(museum_art_gallery_final.sort_values(
    by = ["Museum Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Science Museums are")
display(museum_art_gallery_final.sort_values(
    by = ["Science Museum Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Art Galleries are")
display(museum_art_gallery_final.sort_values(
    by = ["Art Gallery Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Performing Art Venues are")
display(museum_art_gallery_final.sort_values(
    by = ["Performing Arts Venue Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

'Highest number of Art Museums are'

,Neighbourhood,Venue Category,Total Venues,Art Museum Frequency,History Museum Frequency,Museum Frequency,Science Museum Frequency,Art Gallery Frequency,Performing Arts Venue Frequency
146,THE PICTUREHOUSE (THE CATHAY CINEPLEX),Art Museum,2,0.020833,0.020833,0.020833,0.0,0.010417,0.0
26,Consulate of Burkina Faso,Art Museum,2,0.020000,0.000000,0.000000,0.0,0.020000,0.0
45,DRAMA CENTRE,Art Museum,2,0.020000,0.000000,0.000000,0.0,0.010000,0.0
81,MINT MUSEUM OF TOYS,Art Museum,2,0.020000,0.000000,0.000000,0.0,0.000000,0.0
6,BULKHAUL HOUSE,Art Museum,1,0.010000,0.000000,0.000000,0.0,0.000000,0.0


'Highest number of History Museums are'

,Neighbourhood,Venue Category,Total Venues,Art Museum Frequency,History Museum Frequency,Museum Frequency,Science Museum Frequency,Art Gallery Frequency,Performing Arts Venue Frequency
147,THE PICTUREHOUSE (THE CATHAY CINEPLEX),History Museum,2,0.020833,0.020833,0.020833,0.0,0.010417,0.000000
136,STAMFORD PRESCHOOL CHILDCARE MACDONALD HOUSE P...,History Museum,2,0.000000,0.023256,0.023256,0.0,0.000000,0.000000
90,NATIONAL GALLERY SINGAPORE,History Museum,2,0.000000,0.025641,0.000000,0.0,0.025641,0.012821
87,NATIONAL FIRE AND CIVIL EMERGENCY PREPAREDNESS...,History Museum,1,0.000000,0.016949,0.016949,0.0,0.033898,0.000000
134,SRI THENDAYUTHAPANI TEMPLE,History Museum,1,0.000000,0.028571,0.028571,0.0,0.028571,0.000000


'Highest number of General Museums are'

...

,Neighbourhood,Venue Category,Total Venues,Art Museum Frequency,History Museum Frequency,Museum Frequency,Science Museum Frequency,Art Gallery Frequency,Performing Arts Venue Frequency
123,SCIENCE CENTRE BOARD (SCB),Science Museum,1,0.0,0.000000,0.000000,0.066667,0.000000,0.000000
0,ARMENIAN APOSTOLIC CHURCH OF SAINT GREGORY THE...,Art Gallery,1,0.0,0.019231,0.019231,0.000000,0.019231,0.000000
109,PATERSON GARDEN,Art Gallery,1,0.0,0.000000,0.000000,0.000000,0.022222,0.000000
102,OCBC Sentosa,History Museum,1,0.0,0.016129,0.000000,0.000000,0.000000,0.016129
103,OCBC Sentosa,Performing Arts Venue,1,0.0,0.016129,0.000000,0.000000,0.000000,0.016129


'Highest frequency of Art Galleries are'

,Neighbourhood,Venue Category,Total Venues,Art Museum Frequency,History Museum Frequency,Museum Frequency,Science Museum Frequency,Art Gallery Frequency,Performing Arts Venue Frequency
13,CHESTNUT RESIDENCES,Art Gallery,1,0.0,0.0,0.000000,0.0,0.333333,0.0
35,DBS NUS REMIX Branch,Museum,1,0.0,0.0,0.111111,0.0,0.222222,0.0
36,DBS NUS Yusof Ishak House,Art Gallery,2,0.0,0.0,0.111111,0.0,0.222222,0.0
37,DBS NUS Yusof Ishak House,Museum,1,0.0,0.0,0.111111,0.0,0.222222,0.0
34,DBS NUS REMIX Branch,Art Gallery,2,0.0,0.0,0.111111,0.0,0.222222,0.0


'Highest frequency of Performing Art Venues are'

,Neighbourhood,Venue Category,Total Venues,Art Museum Frequency,History Museum Frequency,Museum Frequency,Science Museum Frequency,Art Gallery Frequency,Performing Arts Venue Frequency
48,EMERALD EAST,Performing Arts Venue,1,0.0,0.0,0.0,0.0,0.0,0.055556
4,BMW CENTRE,Performing Arts Venue,1,0.0,0.0,0.0,0.0,0.0,0.043478
33,DBS NTUC Bukit Merah Central,Performing Arts Venue,1,0.0,0.0,0.0,0.0,0.0,0.043478
71,JURONG EAST SPORTS HALL,Performing Arts Venue,1,0.0,0.0,0.0,0.0,0.0,0.040000
29,DBS Defence Technology Tower,Performing Arts Venue,1,0.0,0.0,0.0,0.0,0.0,0.038462


####Shopping Malls and Plazas

In [ ]:
#Let's find neighbourhoods with sizeable presence of Shopping delights!
#There are the ten hand-picked venue categories for Shopping delights -
#“Shopping Mall”, “Shopping Plaza”, “Watch Shop”, “Wine Shop”, “Gourmet Shop”, “Gift Shop”, “Pie Shop”, “Pop-Up Shop”, “Record Shop”.

#First, lets get a list of Neighbourhoods that have at least one such venue, along with the frequency for each category.
shopping_frequency = singapore_grouped[
singapore_grouped["Shopping Mall"] + singapore_grouped["Shopping Plaza"] + singapore_grouped["Watch Shop"]
+ singapore_grouped["Wine Shop"] + singapore_grouped["Gourmet Shop"] + singapore_grouped["Gift Shop"]
 + singapore_grouped["Pie Shop"] + singapore_grouped["Pop-Up Shop"] + singapore_grouped["Record Shop"] > 0]


shopping_frequency = shopping_frequency[["Neighbourhood", "Shopping Mall", "Shopping Plaza", "Watch Shop", "Wine Shop", "Gourmet Shop", "Gift Shop", "Pie Shop", "Pop-Up Shop", "Record Shop"]]
shopping_frequency = shopping_frequency.rename(columns = {"Shopping Mall": "Shopping Mall Frequency",
                                                      "Shopping Plaza": "Shopping Plaza Frequency",
                                                      "Watch Shop": "Watch Shop Frequency",
                                                      "Wine Shop": "Wine Shop Frequency",
                                                      "Gourmet Shop": "Gourmet Shop Frequency",
                                                      "Gift Shop": "Gift Shop Frequency",
                                                      "Pie Shop": "Pie Shop Frequency",
                                                      "Pop-Up Shop": "Pop-Up Shop Frequency",
                                                      "Record Shop": "Record Shop Frequency",})
display("shopping_frequency is")
display(shopping_frequency.head())
display(shopping_frequency.shape)
print("\n")

#Then, we need to get the actual list of these Shops/Malls for each of these neighbourhoods.
shopping = singapore_venues[singapore_venues["Neighbourhood"].isin(shopping_frequency["Neighbourhood"].tolist())]
shopping = shopping[shopping["Venue Category"].isin(["Shopping Mall", "Shopping Plaza", "Watch Shop", "Wine Shop", "Gourmet Shop", "Gift Shop", "Pie Shop", "Pop-Up Shop", "Record Shop"])]
shopping = shopping[["Neighbourhood", "Venue", "Venue Category"]]

display("shopping is")
display(shopping.head())
display(shopping.shape)
print("\n")

#We now have two tables:-
#"shopping_frequency" is a list of Neighbourhoods that have at least one Venue that is a Shop/Mall/Plaza. It also records the frequency of such "Shop/Mall/Plaza" buildings.
#"shopping" is a list of the actual Venues for each Neighbourhood that belong to the category of "Shop/Mall/Plaza".

#Now let's do a GROUP BY to get the unique count of Venues per Neighbourhood and Venue Category.
#This should tell us how many Shops/Malls exist per neighbourhood.
shopping_grouped = shopping.groupby(["Neighbourhood", "Venue Category"])
shopping_grouped = shopping_grouped.agg({"Venue": "nunique"})
shopping_grouped = shopping_grouped.reset_index()
shopping_grouped = shopping_grouped.rename(columns = {"Venue": "Total Venues"})

display("shopping_grouped is")
display(shopping_grouped.head())
display(shopping_grouped.shape)
print("\n")

#Now, "shopping_frequency" will tell us the frequency of Shop/Mall/Plaza in a given Neighbourhood.
#"shopping_grouped" will tell us the actual count of Shop/Mall/Plaza in a given Neighbourhood.
#We can join them to get the entire information in a single table

shopping_final = pd.merge(
    left = shopping_grouped,
    right = shopping_frequency,
    how = "inner",
    on = "Neighbourhood"
)
display("shopping_final is")
display(shopping_final.head())
display(shopping_final.shape)
print("\n")

'shopping_frequency is'

,Neighbourhood,Shopping Mall Frequency,Shopping Plaza Frequency,Watch Shop Frequency,Wine Shop Frequency,Gourmet Shop Frequency,Gift Shop Frequency,Pie Shop Frequency,Pop-Up Shop Frequency,Record Shop Frequency
2,ACEKIDZ @ COMMUNITY,0.032258,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
6,ALEXANDRA NEIGHBOURHOOD POLICE POST,0.000000,0.0,0.035714,0.0,0.0,0.0,0.0,0.0,0.0
14,AMBER GLADES,0.027778,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
22,ANGLICAN CARE CENTRE (SIMEI),0.034483,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
28,ARIVA ON SHAN SERVICED RESIDENCE,0.017241,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


(312, 10)

'shopping is'

,Neighbourhood,Venue,Venue Category
23,MASJID AL-MUTTAQIN,AMK Hub,Shopping Mall
183,TAMIL METHODIST CHURCH,The ZALORA Pop-Up,Pop-Up Shop
192,TAMIL METHODIST CHURCH,Bugis+,Shopping Mall
213,FRAMEWORK BUILDING,PLQ Mall,Shopping Mall
229,FRAMEWORK BUILDING,Paya Lebar Square,Shopping Mall


...

,Neighbourhood,Venue Category,Total Venues
0,ACEKIDZ @ COMMUNITY,Shopping Mall,1
1,ALEXANDRA NEIGHBOURHOOD POLICE POST,Watch Shop,1
2,AMBER GLADES,Shopping Mall,1
3,ANGLICAN CARE CENTRE (SIMEI),Shopping Mall,1
4,ARIVA ON SHAN SERVICED RESIDENCE,Shopping Mall,1


(404, 3)

'shopping_final is'

,Neighbourhood,Venue Category,Total Venues,Shopping Mall Frequency,Shopping Plaza Frequency,Watch Shop Frequency,Wine Shop Frequency,Gourmet Shop Frequency,Gift Shop Frequency,Pie Shop Frequency,Pop-Up Shop Frequency,Record Shop Frequency
0,ACEKIDZ @ COMMUNITY,Shopping Mall,1,0.032258,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,ALEXANDRA NEIGHBOURHOOD POLICE POST,Watch Shop,1,0.000000,0.0,0.035714,0.0,0.0,0.0,0.0,0.0,0.0
2,AMBER GLADES,Shopping Mall,1,0.027778,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,ANGLICAN CARE CENTRE (SIMEI),Shopping Mall,1,0.034483,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,ARIVA ON SHAN SERVICED RESIDENCE,Shopping Mall,1,0.017241,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


(404, 12)

In [ ]:
#AWESOME!
#"shopping_frequency" is a list of Neighbourhoods that have at least one Venue that is a Shop/Mall/Plaza. It also records the frequency of the Shops/Malls/Plazas.
#"shopping" is a list of the actual Venues for each Neighbourhood that belong to the category of Shop/Mall/Plaza.
#"shopping_final" has all the data(well, most) we need.
#In order to show the actual Venue names, we can always go back to "shopping" table.

#Let's see see some Neighbourhoods with the most number of Shop/Mall/Plazas in terms of Count.
display("Highest number of Shopping Malls are")
display(shopping_final[shopping_final["Venue Category"] == "Shopping Mall"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Shopping Plazas are")
display(shopping_final[shopping_final["Venue Category"] == "Shopping Plaza"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Watch Shops are")
display(shopping_final[shopping_final["Venue Category"] == "Watch Shop"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Wine stores are")
display(shopping_final[shopping_final["Venue Category"] == "Wine Shop"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Gourmet shops are")
display(shopping_final[shopping_final["Venue Category"] == "Gourmet Shop"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Gift shops are")
display(shopping_final[shopping_final["Venue Category"] == "Gift Shop"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Pie shops are")
display(shopping_final[shopping_final["Venue Category"] == "Pie Shop"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Pop-Up shops are")
display(shopping_final[shopping_final["Venue Category"] == "Pop-Up Shop"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Record shops are")
display(shopping_final[shopping_final["Venue Category"] == "Record Shop"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

#Let's see see some Neighbourhoods with the most number of Shop/Mall/Plazas in terms of Frequency.
display("Highest frequency of Shopping Malls are")
display(shopping_final.sort_values(
    by = ["Shopping Mall Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Shopping Plazas are")
display(shopping_final.sort_values(
    by = ["Shopping Plaza Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Watch Shops are")
display(shopping_final.sort_values(
    by = ["Watch Shop Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Wine Shops are")
display(shopping_final.sort_values(
    by = ["Wine Shop Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Gourmet Shops are")
display(shopping_final.sort_values(
    by = ["Gourmet Shop Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Gift shops are")
display(shopping_final.sort_values(
    by = ["Gift Shop Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Pie shops are")
display(shopping_final.sort_values(
    by = ["Pie Shop Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Pop-Up shops are")
display(shopping_final.sort_values(
    by = ["Pop-Up Shop Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Record are")
display(shopping_final.sort_values(
    by = ["Record Shop Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

'Highest number of Shopping Malls are'

,Neighbourhood,Venue Category,Total Venues,Shopping Mall Frequency,Shopping Plaza Frequency,Watch Shop Frequency,Wine Shop Frequency,Gourmet Shop Frequency,Gift Shop Frequency,Pie Shop Frequency,Pop-Up Shop Frequency,Record Shop Frequency
28,CATHAY CINELEISURE ORCHARD,Shopping Mall,8,0.080000,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0
251,ORCHARD BUILDING,Shopping Mall,7,0.070000,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0
376,UOB Ngee Ann City,Shopping Mall,7,0.070000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
238,OCBC Paragon,Shopping Mall,7,0.070000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
190,MADRASAH AL-MA'ARIF AL-ISLAMIAH,Shopping Mall,6,0.073171,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


'Highest number of Shopping Plazas are'

,Neighbourhood,Venue Category,Total Venues,Shopping Mall Frequency,Shopping Plaza Frequency,Watch Shop Frequency,Wine Shop Frequency,Gourmet Shop Frequency,Gift Shop Frequency,Pie Shop Frequency,Pop-Up Shop Frequency,Record Shop Frequency
60,COMMUNITY MEDIATION CENTRE (STATE COURTS),Shopping Plaza,1,0.010204,0.010204,0.0,0.00,0.0,0.00,0.0,0.00,0.0
73,Consulate of Burkina Faso,Shopping Plaza,1,0.020000,0.010000,0.0,0.00,0.0,0.01,0.0,0.01,0.0
84,DBS Liang Court,Shopping Plaza,1,0.010000,0.010000,0.0,0.00,0.0,0.01,0.0,0.00,0.0
102,EMBASSY OF MONGOLIA,Shopping Plaza,1,0.020000,0.010000,0.0,0.00,0.0,0.01,0.0,0.01,0.0
111,FRAGRANCE HOTEL - RIVERSIDE,Shopping Plaza,1,0.000000,0.010000,0.0,0.01,0.0,0.01,0.0,0.00,0.0


'Highest number of Watch Shops are'

...

,Neighbourhood,Venue Category,Total Venues,Shopping Mall Frequency,Shopping Plaza Frequency,Watch Shop Frequency,Wine Shop Frequency,Gourmet Shop Frequency,Gift Shop Frequency,Pie Shop Frequency,Pop-Up Shop Frequency,Record Shop Frequency
95,EAST LODGE CLEMENTI,Pie Shop,1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.038462,0.0,0.0
126,HAI LAM SUA TEE KONG TOA TEMPLE,Pie Shop,1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.023810,0.0,0.0
402,ZEH SHING KEONG TEMPLE,Shopping Mall,1,0.014706,0.0,0.000000,0.0,0.0,0.0,0.014706,0.0,0.0
401,ZEH SHING KEONG TEMPLE,Pie Shop,1,0.014706,0.0,0.000000,0.0,0.0,0.0,0.014706,0.0,0.0
266,PCF SPARKLETOTS PRESCHOOL @ BUONA VISTA 32 (CC),Watch Shop,1,0.026316,0.0,0.026316,0.0,0.0,0.0,0.000000,0.0,0.0


'Highest frequency of Pop-Up shops are'

,Neighbourhood,Venue Category,Total Venues,Shopping Mall Frequency,Shopping Plaza Frequency,Watch Shop Frequency,Wine Shop Frequency,Gourmet Shop Frequency,Gift Shop Frequency,Pie Shop Frequency,Pop-Up Shop Frequency,Record Shop Frequency
340,TAMIL METHODIST CHURCH,Shopping Mall,1,0.022222,0.000000,0.0,0.0,0.0,0.000000,0.0,0.022222,0.0
339,TAMIL METHODIST CHURCH,Pop-Up Shop,1,0.022222,0.000000,0.0,0.0,0.0,0.000000,0.0,0.022222,0.0
67,COPYRIGHT TRIBUNAL (MINLAW),Shopping Mall,1,0.012821,0.000000,0.0,0.0,0.0,0.000000,0.0,0.012821,0.0
66,COPYRIGHT TRIBUNAL (MINLAW),Pop-Up Shop,1,0.012821,0.000000,0.0,0.0,0.0,0.000000,0.0,0.012821,0.0
165,KAMPONG GLAM CONSERVATION AREA,Shopping Mall,2,0.025316,0.012658,0.0,0.0,0.0,0.012658,0.0,0.012658,0.0


'Highest frequency of Record are'

,Neighbourhood,Venue Category,Total Venues,Shopping Mall Frequency,Shopping Plaza Frequency,Watch Shop Frequency,Wine Shop Frequency,Gourmet Shop Frequency,Gift Shop Frequency,Pie Shop Frequency,Pop-Up Shop Frequency,Record Shop Frequency
168,KEPPEL DISTRIPARK,Record Shop,1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667
337,TAI YEONG KONG TEMPLE,Record Shop,1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667
243,OCBC Silat Ave - FairPrice,Record Shop,1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090909
161,KAI SAN TEMPLE,Record Shop,1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047619
0,ACEKIDZ @ COMMUNITY,Shopping Mall,1,0.032258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


####A couple looking for good Schools for their kids
A couple are re-locating to Singapore and need our help in identifying suitable neighbourhoods to settle into.

They have a young child, and hence have a priority of finding a good school nearby. They want to stay as close to a good School as possible.

In [24]:

#Let's find neighbourhoods with sizeable presence of Shopping delights!

#These are some of the venue categories corresponding to a  "school" or a "Child Centre".
#"High School", "Martial Arts School", "Music School", "Child Care Service".

#Of these four, the couple would be interested only in the former and the latter. They could choose to send their child to "Music Scool" or "Martial Arts School" as an extra curricular activity
#provided the child shows interest in that field. But for now, let's focus on "High School" and "Child Care Service". The "High School" venue would also have classes for kindergarden kids.

#First, lets get a list of Neighbourhoods that have at least one such venue, along with the frequency for each category.
high_school_frequency = singapore_grouped[singapore_grouped["High School"] + singapore_grouped["Child Care Service"] > 0]


high_school_frequency = high_school_frequency[["Neighbourhood", "High School", "Child Care Service"]]
high_school_frequency = high_school_frequency.rename(columns = {"High School": "High School Frequency",
                                                      "Child Care Service": "Child Care Service Frequency"})
display("high_school_frequency is")
display(high_school_frequency.head())
display(high_school_frequency.shape)
print("\n")

#Then, we need to get the actual list of these Schools/Services for each of these neighbourhoods.
high_school = singapore_venues[singapore_venues["Neighbourhood"].isin(high_school_frequency["Neighbourhood"].tolist())]
shopping = high_school[high_school["Venue Category"].isin(["High School", "Child Care Service"])]
high_school = high_school[["Neighbourhood", "Venue", "Venue Category"]]

display("high_school is")
display(high_school.head())
display(high_school.shape)
print("\n")

#We now have two tables:-
#"high_school_frequency" is a list of Neighbourhoods that have at least one Venue that is a School/Child Care. It also records the frequency of such "School/Child Care" buildings.
#"high_school" is a list of the actual Venues for each Neighbourhood that belong to the category of "School/Child Care".

#Now let's do a GROUP BY to get the unique count of Venues per Neighbourhood and Venue Category.
#This should tell us how many Shops/Malls exist per neighbourhood.
high_school_grouped = high_school.groupby(["Neighbourhood", "Venue Category"])
high_school_grouped = high_school_grouped.agg({"Venue": "nunique"})
high_school_grouped = high_school_grouped.reset_index()
high_school_grouped = high_school_grouped.rename(columns = {"Venue": "Total Venues"})

display("high_school_grouped is")
display(high_school_grouped.head())
display(high_school_grouped.shape)
print("\n")

#Now, "high_school_frequency" will tell us the frequency of "School/Child Care" in a given Neighbourhood.
#"high_school_grouped" will tell us the actual count of School/Child Care in a given Neighbourhood.
#We can join them to get the entire information in a single table

high_school_final = pd.merge(
    left = high_school_grouped,
    right = high_school_frequency,
    how = "inner",
    on = "Neighbourhood"
)
display("high_school_final is")
display(high_school_final.head())
display(high_school_final.shape)
print("\n")

'high_school_frequency is'

,Neighbourhood,High School Frequency,Child Care Service Frequency
105,CELADON VIEW,0.033333,0.000000
584,PASIR RIS CENTRAL HAWKER CENTRE,0.022727,0.000000
632,Pro-Teach Schoolcare (Wellington),0.000000,0.100000
653,RIVERLIFE CHURCH,0.033333,0.000000
724,SIXTH AVENUE VILLE,0.000000,0.034483


(6, 3)

'high_school is'

,Neighbourhood,Venue,Venue Category
1115,Pro-Teach Schoolcare (Wellington),White Restaurant,Chinese Restaurant
1116,Pro-Teach Schoolcare (Wellington),McDonald's,Fast Food Restaurant
1117,Pro-Teach Schoolcare (Wellington),Little Jimmy,Wings Joint
1118,Pro-Teach Schoolcare (Wellington),588C Basketball Court,Basketball Court
1119,Pro-Teach Schoolcare (Wellington),Fitness Corner @ Admiralty Drive,Park


...

,Neighbourhood,Venue Category,Total Venues
0,CELADON VIEW,American Restaurant,1
1,CELADON VIEW,Asian Restaurant,1
2,CELADON VIEW,Beach,1
3,CELADON VIEW,Bubble Tea Shop,1
4,CELADON VIEW,Buffet,1


(123, 3)

'high_school_final is'

,Neighbourhood,Venue Category,Total Venues,High School Frequency,Child Care Service Frequency
0,CELADON VIEW,American Restaurant,1,0.033333,0.0
1,CELADON VIEW,Asian Restaurant,1,0.033333,0.0
2,CELADON VIEW,Beach,1,0.033333,0.0
3,CELADON VIEW,Bubble Tea Shop,1,0.033333,0.0
4,CELADON VIEW,Buffet,1,0.033333,0.0


(123, 5)

In [25]:
#AWESOME!
#"high_school_frequency" is a list of Neighbourhoods that have at least one Venue that is a School/Child Care. It also records the frequency of the Schools/Child Cares.
#"high_school" is a list of the actual Venues for each Neighbourhood that belong to the category of School/Child Care.
#"high_school_final" has all the data(well, most) we need.
#In order to show the actual Venue names, we can always go back to "high_school" table.

#Let's see see some Neighbourhoods with the most number of School/Child Care in terms of Count.
display("Highest number of Schools are")
display(high_school_final[high_school_final["Venue Category"] == "High School"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Child Care Services are")
display(high_school_final[high_school_final["Venue Category"] == "Child Care Service"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

#Let's see see some Neighbourhoods with the most number of Schools/Child Cares in terms of Frequency.
display("Highest frequency of High Schools are")
display(high_school_final.sort_values(
    by = ["High School Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Child Care Services are")
display(high_school_final.sort_values(
    by = ["Child Care Service Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

'Highest number of Schools are'

,Neighbourhood,Venue Category,Total Venues,High School Frequency,Child Care Service Frequency
10,CELADON VIEW,High School,1,0.033333,0.0
40,PASIR RIS CENTRAL HAWKER CENTRE,High School,1,0.022727,0.0
77,RIVERLIFE CHURCH,High School,1,0.033333,0.0
119,THE PERIWINKLE,High School,1,0.111111,0.0


'Highest number of Child Care Services are'

,Neighbourhood,Venue Category,Total Venues,High School Frequency,Child Care Service Frequency
59,Pro-Teach Schoolcare (Wellington),Child Care Service,1,0.0,0.100000
99,SIXTH AVENUE VILLE,Child Care Service,1,0.0,0.034483


'Highest frequency of High Schools are'

,Neighbourhood,Venue Category,Total Venues,High School Frequency,Child Care Service Frequency
122,THE PERIWINKLE,Supermarket,1,0.111111,0.0
121,THE PERIWINKLE,Steakhouse,1,0.111111,0.0
120,THE PERIWINKLE,Moving Target,1,0.111111,0.0
119,THE PERIWINKLE,High School,1,0.111111,0.0
118,THE PERIWINKLE,Food,1,0.111111,0.0


'Highest frequency of Child Care Services are'

,Neighbourhood,Venue Category,Total Venues,High School Frequency,Child Care Service Frequency
61,Pro-Teach Schoolcare (Wellington),Fast Food Restaurant,1,0.0,0.1
65,Pro-Teach Schoolcare (Wellington),Sporting Goods Shop,1,0.0,0.1
63,Pro-Teach Schoolcare (Wellington),Park,1,0.0,0.1
62,Pro-Teach Schoolcare (Wellington),Grocery Store,1,0.0,0.1
66,Pro-Teach Schoolcare (Wellington),Wings Joint,1,0.0,0.1


####A young professional looking for good Chinese Restaurants and Movei Theaters

In [40]:

#Let's find neighbourhoods with sizeable presence of Theaters and Movie Theatres!

#These are some of the venue categories corresponding to Theaters
#"Movie Theater", "Theater", "Performing Arts Venue"

#These are some of the venue categories corresponding to Chinese Restaurants
#"Cantonese Restaurant", "Chinese Restaurant", "Hainan Restaurant", "Hakka Restaurant", "Dim Sum Restaurant"


#We need to find neighbourhoods that have both the above list of venues - "Theatres" and "Chinese Restaurants"

#First, lets get a list of Neighbourhoods that have at least one Theatre.
theater_frequency = singapore_grouped[
singapore_grouped["Movie Theater"] + singapore_grouped["Theater"] +  singapore_grouped["Performing Arts Venue"] > 0]

theater_frequency = theater_frequency[["Neighbourhood", "Movie Theater", "Theater", "Performing Arts Venue"]]
theater_frequency = theater_frequency.rename(columns = {"Movie Theater": "Movie Theater Frequency",
                                                      "Theater": "Theater Frequency",
                                                      "Performing Arts Venue": "Performing Arts Venue Frequency"})

#Then, lets repeat the same for Neighbourhoods that have at least one Chinese Restaurant.
chinese_restaurant_frequency = singapore_grouped[
singapore_grouped["Cantonese Restaurant"] + singapore_grouped["Chinese Restaurant"] + singapore_grouped["Hainan Restaurant"] + singapore_grouped["Hakka Restaurant"] + singapore_grouped["Dim Sum Restaurant"] > 0]

chinese_restaurant_frequency = chinese_restaurant_frequency[["Neighbourhood", "Cantonese Restaurant", "Chinese Restaurant", "Hainan Restaurant", "Hakka Restaurant", "Dim Sum Restaurant"]]
chinese_restaurant_frequency = chinese_restaurant_frequency.rename(columns = {"Cantonese Restaurant": "Cantonese Restaurant Frequency",
                                                      "Chinese Restaurant": "Chinese Restaurant Frequency",
                                                      "Hainan Restaurant": "Hainan Restaurant Frequency",
                                                      "Hakka Restaurant": "Hakka Restaurant Frequency",
                                                      "Dim Sum Restaurant": "Dim Sum Restaurant Frequency"})

#Now, we have two seperate tables - Theaters and Chinese Restaurants.
#We need to Merge them to find the Neighbourhoods that have at least one of both.
theatre_chinese_restaurant_frequency = pd.merge(
    left = theater_frequency,
    right = chinese_restaurant_frequency,
    how = "inner",
    on = "Neighbourhood"
)

display("theatre_chinese_restaurant_frequency is")
display(theatre_chinese_restaurant_frequency.head())
display(theatre_chinese_restaurant_frequency.shape)
print("There are " + str(theatre_chinese_restaurant_frequency.Neighbourhood.nunique()) + " unique Neighbourhoods that have both a Theater and Chinese Restaurant")
print("\n")

#Going forward, we will use this "theatre_chinese_restaurant_frequency" table instead of the "theater_frequency" and "chinese_restaurant_frequency" tables

#Now, we need to get the actual list of these Theatres/Restaurants for each of these neighbourhoods.
theatre_chinese_restaurant = singapore_venues[singapore_venues["Neighbourhood"].isin(theatre_chinese_restaurant_frequency["Neighbourhood"].tolist())]
theatre_chinese_restaurant = theatre_chinese_restaurant[theatre_chinese_restaurant["Venue Category"].isin(["Movie Theater", "Theater", "Performing Arts Venue", "Dim Sum Restaurant",
                                                                                                           "Hakka Restaurant", "Hainan Restaurant", "Cantonese Restaurant", "Chinese Restaurant"])]
theatre_chinese_restaurant = theatre_chinese_restaurant[["Neighbourhood", "Venue", "Venue Category"]]


display("theatre_chinese_restaurant is")
display(theatre_chinese_restaurant.head())
display(theatre_chinese_restaurant.shape)
print("\n")

#We now have two tables:-
#"theatre_chinese_restaurant_frequency" is a list of Neighbourhoods that have at least one Venue that in the category of "Theaters" and "Chinese Restaurant". It also records the frequency of such venues.
#"theatre_chinese_restaurant" is a list of the actual Venues for each Neighbourhood that belong to the category of "Theaters" and "Chinese Restaurant".

#Now let's do a GROUP BY to get the unique count of Venues per Neighbourhood and Venue Category.
#This should tell us how many Theatres and Chinese Restaurants exist per neighbourhood.
theatre_chinese_restaurant_grouped = theatre_chinese_restaurant.groupby(["Neighbourhood", "Venue Category"])
theatre_chinese_restaurant_grouped = theatre_chinese_restaurant_grouped.agg({"Venue": "nunique"})
theatre_chinese_restaurant_grouped = theatre_chinese_restaurant_grouped.reset_index()
theatre_chinese_restaurant_grouped = theatre_chinese_restaurant_grouped.rename(columns = {"Venue": "Total Venues"})

display("theatre_chinese_restaurant_grouped is")
display(theatre_chinese_restaurant_grouped.head())
display(theatre_chinese_restaurant_grouped.shape)
print("\n")

#Now, "theatre_chinese_restaurant_frequency" will tell us the frequency of "Theatres" and "Chinese Restaurants" in a given Neighbourhood.
#"theatre_chinese_restaurant_grouped" will tell us the actual count of such venues in a given Neighbourhood.
#We can join them to get the entire information in a single table

theatre_chinese_restaurant_final = pd.merge(
    left = theatre_chinese_restaurant_grouped,
    right = theatre_chinese_restaurant_frequency,
    how = "inner",
    on = "Neighbourhood"
)
display("theatre_chinese_restaurant_final is")
display(theatre_chinese_restaurant_final.head())
display(theatre_chinese_restaurant_final.shape)
print("\n")

'theatre_chinese_restaurant_frequency is'

,Neighbourhood,Movie Theater Frequency,Theater Frequency,Performing Arts Venue Frequency,Cantonese Restaurant Frequency,Chinese Restaurant Frequency,Hainan Restaurant Frequency,Hakka Restaurant Frequency,Dim Sum Restaurant Frequency
0,ASIAN CIVILISATIONS MUSEUM BOARD,0.000000,0.0125,0.000000,0.0,0.012500,0.0,0.0,0.000000
1,BALMORAL TOWER,0.125000,0.0000,0.000000,0.0,0.062500,0.0,0.0,0.000000
2,BEACH ARRIVAL PLAZA,0.016129,0.0000,0.000000,0.0,0.016129,0.0,0.0,0.000000
3,BRIGHTON MONTESSORI INTERNATIONAL PRE-SCHOOL @...,0.000000,0.0000,0.076923,0.0,0.076923,0.0,0.0,0.076923
4,CAMBERLEY INTERNATIONAL PRE-SCHOOL PRIVATE LIM...,0.125000,0.0000,0.000000,0.0,0.062500,0.0,0.0,0.000000


(77, 9)

There are 77 unique Neighbourhoods that have both a Theater and Chinese Restaurant




'theatre_chinese_restaurant is'

...

,Neighbourhood,Venue Category,Total Venues
0,ASIAN CIVILISATIONS MUSEUM BOARD,Chinese Restaurant,1
1,ASIAN CIVILISATIONS MUSEUM BOARD,Theater,1
2,BALMORAL TOWER,Chinese Restaurant,1
3,BALMORAL TOWER,Movie Theater,2
4,BEACH ARRIVAL PLAZA,Chinese Restaurant,1


(196, 3)

'theatre_chinese_restaurant_final is'

,Neighbourhood,Venue Category,Total Venues,Movie Theater Frequency,Theater Frequency,Performing Arts Venue Frequency,Cantonese Restaurant Frequency,Chinese Restaurant Frequency,Hainan Restaurant Frequency,Hakka Restaurant Frequency,Dim Sum Restaurant Frequency
0,ASIAN CIVILISATIONS MUSEUM BOARD,Chinese Restaurant,1,0.000000,0.0125,0.0,0.0,0.012500,0.0,0.0,0.0
1,ASIAN CIVILISATIONS MUSEUM BOARD,Theater,1,0.000000,0.0125,0.0,0.0,0.012500,0.0,0.0,0.0
2,BALMORAL TOWER,Chinese Restaurant,1,0.125000,0.0000,0.0,0.0,0.062500,0.0,0.0,0.0
3,BALMORAL TOWER,Movie Theater,2,0.125000,0.0000,0.0,0.0,0.062500,0.0,0.0,0.0
4,BEACH ARRIVAL PLAZA,Chinese Restaurant,1,0.016129,0.0000,0.0,0.0,0.016129,0.0,0.0,0.0


(196, 11)

In [42]:
#AWESOME!
#"theatre_chinese_restaurant_frequency" is a list of Neighbourhoods that have at least one Venue from both "Theater" and "Chinese Restaurant" category. It also records the frequency of such Venues.
#"theatre_chinese_restaurant" is a list of the actual Venues for each Neighbourhood that belong to the category of "Theatre" or "Chinese Restaurant".
#"theatre_chinese_restaurant_final" has all the data(well, most) we need.
#In order to show the actual Venue names, we can always go back to "theatre_chinese_restaurant" table.


#"Movie Theater", "Theater", "Performing Arts Venue", "Dim Sum Restaurant", "Hakka Restaurant", "Hainan Restaurant", "Cantonese Restaurant", "Chinese Restaurant"


#Let's see see some Neighbourhoods with the most number of Theater/Chinese Restaurant in terms of Count.
display("Highest number of Movie Theaters are")
display(theatre_chinese_restaurant_final[theatre_chinese_restaurant_final["Venue Category"] == "Movie Theater"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Theatres are")
display(theatre_chinese_restaurant_final[theatre_chinese_restaurant_final["Venue Category"] == "Theater"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Performing Arts Venues are")
display(theatre_chinese_restaurant_final[theatre_chinese_restaurant_final["Venue Category"] == "Performing Arts Venue"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Dim Sum Restaurant are")
display(theatre_chinese_restaurant_final[theatre_chinese_restaurant_final["Venue Category"] == "Dim Sum Restaurant"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Hakka Restaurant are")
display(theatre_chinese_restaurant_final[theatre_chinese_restaurant_final["Venue Category"] == "Hakka Restaurant"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Hainan Restaurant are")
display(high_school_final[high_school_final["Venue Category"] == "Hainan Restaurant"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Cantonese Restaurant are")
display(theatre_chinese_restaurant_final[theatre_chinese_restaurant_final["Venue Category"] == "Cantonese Restaurant"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest number of Chinese Restaurant are")
display(theatre_chinese_restaurant_final[theatre_chinese_restaurant_final["Venue Category"] == "Chinese Restaurant"].sort_values(
    by = ["Total Venues"],
    inplace = False,
    ascending = False
).head())
print("\n\n")


#Let's see see some Neighbourhoods with the most number of Theatres/Chinese Restaurants in terms of Frequency.
display("Highest frequency of Movie Theatres are")
display(theatre_chinese_restaurant_final.sort_values(
    by = ["Movie Theater Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Theatres are")
display(theatre_chinese_restaurant_final.sort_values(
    by = ["Theater Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Dim Sum Restaurants are")
display(theatre_chinese_restaurant_final.sort_values(
    by = ["Dim Sum Restaurant Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Cantonese Restaurants are")
display(theatre_chinese_restaurant_final.sort_values(
    by = ["Cantonese Restaurant Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Chinese Restaurants are")
display(theatre_chinese_restaurant_final.sort_values(
    by = ["Chinese Restaurant Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Hainan Restaurants are")
display(theatre_chinese_restaurant_final.sort_values(
    by = ["Hainan Restaurant Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

display("Highest frequency of Hakka Restaurants are")
display(theatre_chinese_restaurant_final.sort_values(
    by = ["Hakka Restaurant Frequency"],
    inplace = False,
    ascending = False
).head())
print("\n\n")

'Highest number of Movie Theaters are'

,Neighbourhood,Venue Category,Total Venues,Movie Theater Frequency,Theater Frequency,Performing Arts Venue Frequency,Cantonese Restaurant Frequency,Chinese Restaurant Frequency,Hainan Restaurant Frequency,Hakka Restaurant Frequency,Dim Sum Restaurant Frequency
129,SINGAPORE MANAGEMENT UNIVERSITY (SCH OF ECONOM...,Movie Theater,3,0.030000,0.02,0.0,0.0,0.000000,0.0,0.0,0.01
3,BALMORAL TOWER,Movie Theater,2,0.125000,0.00,0.0,0.0,0.062500,0.0,0.0,0.00
54,GOODWOOD GRAND,Movie Theater,2,0.105263,0.00,0.0,0.0,0.052632,0.0,0.0,0.00
161,THE BALMORAL SPRING,Movie Theater,2,0.090909,0.00,0.0,0.0,0.090909,0.0,0.0,0.00
10,CAMBERLEY INTERNATIONAL PRE-SCHOOL PRIVATE LIM...,Movie Theater,2,0.125000,0.00,0.0,0.0,0.062500,0.0,0.0,0.00


'Highest number of Theatres are'

,Neighbourhood,Venue Category,Total Venues,Movie Theater Frequency,Theater Frequency,Performing Arts Venue Frequency,Cantonese Restaurant Frequency,Chinese Restaurant Frequency,Hainan Restaurant Frequency,Hakka Restaurant Frequency,Dim Sum Restaurant Frequency
171,UNIVERSAL STUDIOS SINGAPORE THEME PARK,Theater,2,0.010000,0.020000,0.020000,0.000000,0.020000,0.0,0.0,0.000000
156,THE ADELPHI,Theater,2,0.013699,0.027397,0.013699,0.013699,0.013699,0.0,0.0,0.013699
141,SOUTH BEACH,Theater,2,0.000000,0.020000,0.010000,0.010000,0.010000,0.0,0.0,0.020000
130,SINGAPORE MANAGEMENT UNIVERSITY (SCH OF ECONOM...,Theater,2,0.030000,0.020000,0.000000,0.000000,0.000000,0.0,0.0,0.010000
1,ASIAN CIVILISATIONS MUSEUM BOARD,Theater,1,0.000000,0.012500,0.000000,0.000000,0.012500,0.0,0.0,0.000000


'Highest number of Performing Arts Venues are'

...

,Neighbourhood,Venue Category,Total Venues,Movie Theater Frequency,Theater Frequency,Performing Arts Venue Frequency,Cantonese Restaurant Frequency,Chinese Restaurant Frequency,Hainan Restaurant Frequency,Hakka Restaurant Frequency,Dim Sum Restaurant Frequency
92,MARKET & HAWKER CENTRE (BLK 20 GHIM MOH ROAD),Performing Arts Venue,1,0.0,0.0,0.031250,0.0,0.218750,0.000000,0.0,0.0
91,MARKET & HAWKER CENTRE (BLK 20 GHIM MOH ROAD),Chinese Restaurant,7,0.0,0.0,0.031250,0.0,0.218750,0.000000,0.0,0.0
43,ENABLING VILLAGE,Chinese Restaurant,5,0.0,0.0,0.035714,0.0,0.178571,0.035714,0.0,0.0
44,ENABLING VILLAGE,Hainan Restaurant,1,0.0,0.0,0.035714,0.0,0.178571,0.035714,0.0,0.0
45,ENABLING VILLAGE,Performing Arts Venue,1,0.0,0.0,0.035714,0.0,0.178571,0.035714,0.0,0.0


'Highest frequency of Hainan Restaurants are'

,Neighbourhood,Venue Category,Total Venues,Movie Theater Frequency,Theater Frequency,Performing Arts Venue Frequency,Cantonese Restaurant Frequency,Chinese Restaurant Frequency,Hainan Restaurant Frequency,Hakka Restaurant Frequency,Dim Sum Restaurant Frequency
45,ENABLING VILLAGE,Performing Arts Venue,1,0.000000,0.0,0.035714,0.000000,0.178571,0.035714,0.0,0.0
44,ENABLING VILLAGE,Hainan Restaurant,1,0.000000,0.0,0.035714,0.000000,0.178571,0.035714,0.0,0.0
43,ENABLING VILLAGE,Chinese Restaurant,5,0.000000,0.0,0.035714,0.000000,0.178571,0.035714,0.0,0.0
41,EMERALD PARK,Hainan Restaurant,1,0.014706,0.0,0.000000,0.014706,0.147059,0.014706,0.0,0.0
42,EMERALD PARK,Movie Theater,1,0.014706,0.0,0.000000,0.014706,0.147059,0.014706,0.0,0.0


'Highest frequency of Hakka Restaurants are'

,Neighbourhood,Venue Category,Total Venues,Movie Theater Frequency,Theater Frequency,Performing Arts Venue Frequency,Cantonese Restaurant Frequency,Chinese Restaurant Frequency,Hainan Restaurant Frequency,Hakka Restaurant Frequency,Dim Sum Restaurant Frequency
0,ASIAN CIVILISATIONS MUSEUM BOARD,Chinese Restaurant,1,0.000000,0.0125,0.0,0.0,0.012500,0.0,0.0,0.0
123,ROYAL PLAZA,Movie Theater,1,0.010000,0.0000,0.0,0.0,0.040000,0.0,0.0,0.0
125,RV SUITES,Movie Theater,1,0.026316,0.0000,0.0,0.0,0.052632,0.0,0.0,0.0
126,SEMBAWANG TOWN COUNCIL (WOODLANDS COLLECTION C...,Chinese Restaurant,2,0.018182,0.0000,0.0,0.0,0.036364,0.0,0.0,0.0
127,SEMBAWANG TOWN COUNCIL (WOODLANDS COLLECTION C...,Movie Theater,1,0.018182,0.0000,0.0,0.0,0.036364,0.0,0.0,0.0
